In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPooling2D, LSTM, Dropout, BatchNormalization
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam

# Function to calculate Mean Bias Error (MBE)
def mean_bias_error(y_true, y_pred):
    return np.mean(y_true - y_pred)

# Function to calculate Mean Absolute Percentage Error (MAPE)
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Read the CSV file
file_name = 'SeaData\\Cold_Tongue_Equatorial Current_s_dbias.csv'
df = pd.read_csv(file_name, parse_dates=['date'], index_col='date')

# Fill in missing data using forward filling
df.fillna(method='ffill', inplace=True)

# Apply STL decomposition to remove seasonality and trends
from statsmodels.tsa.seasonal import STL
stl = STL(df['dbiastg'], seasonal=7)
res = stl.fit()
df['residual'] = res.resid

# Create a Time Series Imaging representation (sliding window)
def create_images(time_series, window_size):
    images = []
    for i in range(len(time_series) - window_size):
        image = time_series[i:i + window_size].values
        images.append(image)
    return np.array(images)

# Set the window size for the time series images (adjust as needed)
window_size = 30
time_series_images = create_images(df['residual'], window_size)

# Reshape the data to simulate 2D images (we can treat time series as single-channel images)
X = time_series_images.reshape(time_series_images.shape[0], window_size, 1, 1)

# Split into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, df['residual'][window_size:].values, test_size=0.2, shuffle=False)

# Build an enhanced CNN + LSTM model with Dropout and BatchNormalization
model = Sequential()

# CNN Layers
model.add(Conv2D(64, (3, 1), activation='relu', input_shape=(window_size, 1, 1)))
model.add(MaxPooling2D((2, 1)))
model.add(Dropout(0.3))  # Dropout to prevent overfitting
model.add(BatchNormalization())

# LSTM Layer for time dependencies
model.add(Flatten())
model.add(LSTM(64, return_sequences=False))  # Captures time dependencies

# Dense Layers for final prediction
model.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.3))  # Another Dropout layer
model.add(Dense(1))  # Output Layer

# Compile the model with Adam optimizer and learning rate tuning
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))

# Forecasting (predicting the next values based on learned patterns)
forecast_steps = 10
forecast = []
current_input = df['residual'][-window_size:].values.reshape(1, window_size, 1, 1)

for _ in range(forecast_steps):
    pred = model.predict(current_input)
    forecast.append(pred[0][0])
    current_input = np.roll(current_input, -1, axis=1)  # Slide the window
    current_input[0, -1, 0, 0] = pred  # Update the last value with the prediction

# Add the seasonal and trend components back to the forecast
forecast = np.array(forecast) + res.seasonal[-forecast_steps:] + res.trend[-forecast_steps:]

# Plot the original data and the forecast
plt.figure(figsize=(10, 5))
plt.plot(df.index[-100:], df['dbiastg'].iloc[-100:], label='Original Data')
plt.plot(df.index[-10:], forecast, label='Forecast', linestyle='--', color='red')
plt.xlabel('Date')
plt.ylabel('dbiastg')
plt.title('Time Series Data with Enhanced Time Series Imaging Forecast')
plt.legend()
plt.show()

# Calculate performance metrics
rmse = np.sqrt(mean_squared_error(df['dbiastg'][-10:], forecast))
mae = mean_absolute_error(df['dbiastg'][-10:], forecast)
mbe = mean_bias_error(df['dbiastg'][-10:], forecast)
mape = mean_absolute_percentage_error(df['dbiastg'][-10:], forecast)

print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"MBE: {mbe}")
print(f"MAPE: {mape}")
